<a href="https://colab.research.google.com/github/cbonnin88/meditrack/blob/main/amplitude_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import json
import time
from datetime import datetime

In [ ]:
# 1. SETUP
api_key = 'e3bf6d0ce5e1daeb582693a004bc9218'
url ='https://api.eu.amplitude.com/2/httpapi'

In [ ]:
# 2. LOAD DATA
df_events = pd.read_csv('meditrack_events_clean.csv')

In [ ]:
# 2. HELPER FUNCTION: Convert Date to Unix Timestampe (Miliseconds)
def to_unix_ms(date_str):
  dt = datetime.strptime(date_str, '%Y-%m-%d')
  return int(dt.timestamp() * 1000)

In [ ]:
# 4. PREPARE THE PAYLOAD
events_buffer = []
print(f'Preparing {len(df_events)} events for upload')

for _, row in df_events.iterrows():
  event = {
      'user_id': str(row['user_id']),
      'event_type': row['event_name'],
      'time': to_unix_ms(row['event_timestamp']),
      'platform': row['platform'],
      'event_properties':{
          'source':'MediTrack_Data_Pipeline',
          'platform':row['platform']
      }
  }
  events_buffer.append(event)

Preparing 6382 events for upload


In [ ]:
# 5. SEND DATA (In Batches of 1000 to be safe)
batch_size = 1000
for i in range(0, len(events_buffer), batch_size):
  batch = events_buffer[i:i + batch_size]

  payload = {
      'api_key':api_key,
      'events': batch
  }

  # Sending Request
  response = requests.post(url, json=payload)

  if response.status_code == 200:
    print(f'✅ Batch {i//batch_size + 1} uploaded successfully !')
  else:
    print(f'❌ Error uploading batch: {response.text}')

  time.sleep(1) # Be nice to the API

print('Data Ingestion Complete. Go Check Amplitude')

✅ Batch 1 uploaded successfully !
✅ Batch 2 uploaded successfully !
✅ Batch 3 uploaded successfully !
✅ Batch 4 uploaded successfully !
✅ Batch 5 uploaded successfully !
✅ Batch 6 uploaded successfully !
✅ Batch 7 uploaded successfully !
Data Ingestion Complete. Go Check Amplitude
